In [1]:
import numpy as np
import call_my_bluff as cmb

# Bet

Dice are 0, 1, 2, 3, 4 and 5 (6-sided dice) 
5 represents the star side

A bet is represented by num_dice and dice_value

Each bet also has an index with 0 corresponding to the lowest bet you can make and 109 is the highest bet you can make

In [6]:
bet = cmb.game.Bet(num_dice=2, dice_value=3)
print("num_dice:", bet.num_dice)
print("dice_value:", bet.dice_value)
print("index:", bet.index)

num_dice: 2
dice_value: 3
index: 9


In [9]:
bet.index = 78
print("num_dice:", bet.num_dice)
print("dice_value:", bet.dice_value)
print("index:", bet.index)

num_dice: 15
dice_value: 1
index: 78


In [10]:
bet.num_dice = 4
print("num_dice:", bet.num_dice)
print("dice_value:", bet.dice_value)
print("index:", bet.index)

num_dice: 4
dice_value: 1
index: 18


In [8]:
for i in range(33, 45):
    bet = cmb.game.Bet(index=i)
    print(f"index: {i}, num_dice: {bet.num_dice}, dice_value: {bet.dice_value}")

index: 33, num_dice: 7, dice_value: 0
index: 34, num_dice: 7, dice_value: 1
index: 35, num_dice: 7, dice_value: 2
index: 36, num_dice: 7, dice_value: 3
index: 37, num_dice: 7, dice_value: 4
index: 38, num_dice: 4, dice_value: 5
index: 39, num_dice: 8, dice_value: 0
index: 40, num_dice: 8, dice_value: 1
index: 41, num_dice: 8, dice_value: 2
index: 42, num_dice: 8, dice_value: 3
index: 43, num_dice: 8, dice_value: 4
index: 44, num_dice: 9, dice_value: 0


# Game State

In [3]:
'''
@dataclass
class State:
    num_players: int
    bet: Bet
    player_curr: int
    player_prev: Optional[int]
    turn_order: List[int]
    num_dice: List[int]
    dice: List[List[int]]
    dice_locked: List[List[bool]]
    action_log: List[Action]
'''

In [4]:
state = cmb.game.initialize_game(3)
state

State(num_players=3, bet=<call_my_bluff.game.Bet object at 0x119be2450>, player_curr=0, player_prev=None, turn_order=[0, 1, 2], num_dice=[5, 5, 5], dice=[[3, 3, 1, 4, 4], [5, 5, 2, 0, 0], [3, 1, 4, 1, 2]], dice_locked=[[False, False, False, False, False], [False, False, False, False, False], [False, False, False, False, False]], action_log=[])

# Observation

In [ ]:
'''
@dataclass
class Observation:
    player: int
    bet: Bet
    unknown_dice: List[int]
    known_dice: List[List[int]]
    player_locked_dice: List[bool]
    action_log: List[Action]
'''

In [14]:
observation = cmb.game.player_observation(state)
observation

Observation(player=0, bet=<call_my_bluff.game.Bet object at 0x119be2450>, unknown_dice=[0, 5, 5], known_dice=[[3, 3, 1, 4, 4], [], []], player_locked_dice=[False, False, False, False, False], action_log=[])

# Action

In [ ]:
'''
@dataclass
class Action:
    type: ActionType
    dice_to_lock: Optional[List[bool]] = None
    bet: Optional[Bet] = None
    result: Optional[Tuple[int]] = None
    player: Optional[int] = None

class ActionType(Enum):
    CALL = 0
    BET = 1
    REROLL_BET = 2
    RESULT = 3
'''

In [16]:
# Call action
action = cmb.game.Action(type=cmb.game.ActionType.CALL)

# Bet action
action = cmb.game.Action(type=cmb.game.ActionType.BET, bet=cmb.game.Bet(num_dice=2, dice_value=3))

# Reroll bet action
action = cmb.game.Action(type=cmb.game.ActionType.REROLL_BET, 
                         bet=cmb.game.Bet(num_dice=2, dice_value=3),
                         dice_to_lock=[True, False, False, False, False])

# Result action
# handled by the game

In [17]:
state = cmb.game.player_action(state, action)
state

State(num_players=3, bet=<call_my_bluff.game.Bet object at 0x11e869910>, player_curr=1, player_prev=0, turn_order=[0, 1, 2], num_dice=[5, 5, 5], dice=[[3, 2, 1, 3, 0], [5, 5, 2, 0, 0], [3, 1, 4, 1, 2]], dice_locked=[[True, False, False, False, False], [False, False, False, False, False], [False, False, False, False, False]], action_log=[Action(type=<ActionType.REROLL_BET: 2>, dice_to_lock=[True, False, False, False, False], bet=<call_my_bluff.game.Bet object at 0x11e869910>, result=None, player=0)])

# Example game

In [18]:
agents = [cmb.agents.MaxAgent(), cmb.agents.SimpleAgent(), cmb.agents.SimpleAgent()]
state = cmb.game.initialize_game(len(agents))

while not cmb.game.game_over(state):
    while not cmb.game.round_over(state):

        observation = cmb.game.player_observation(state)
        action = agents[state.player_curr].policy(observation)
        state = cmb.game.player_action(state, action)
        cmb.game.render(state)

    for agent in agents:
        agent.round_results(state)
    state = cmb.game.new_round(state)



####################
New round started.
Players have [5, 5, 5] dice.
--------------------
Player 2's turn, their dice are: [4, 3, 5, 1, 5]
Player 2 bet 4 0s
--------------------
Player 1's turn, their dice are: [4, 4, 0, 0, 2]
Player 1 bet 4 4s
--------------------
Player 0's turn, their dice are: [5, 1, 0, 0, 3]
Player 0 bet 6 0s
--------------------
Player 2's turn, their dice are: [4, 3, 5, 1, 5]
Player 2 called the bet.
There are actually 7 0s.
Player 2 loses 1 dice.
####################
New round started.
Players have [5, 5, 4] dice.
--------------------
Player 0's turn, their dice are: [3, 3, 1, 0, 2]
Player 0 bet 5 3s
--------------------
Player 2's turn, their dice are: [3, 0, 5, 0]
Player 2 called the bet.
There are actually 5 3s.
Everyone but the better loses a die.
####################
New round started.
Players have [5, 4, 3] dice.
--------------------
Player 0's turn, their dice are: [4, 2, 1, 1, 1]
Player 0 bet 5 1s
--------------------
Player 2's turn, their dice are: [5